This notebook creates TIFF images from the environmental products that you can use to create maps.

You need Google Earth Engine to retrieve the temperature and vegetation images. These will be saved to your Google Drive, from where you can download them.

In [1]:
import ee
import numpy as np
import pandas as pd
import geemap
import geopandas as gpd
import netCDF4
from shapely.geometry import mapping
from datetime import datetime, date, timedelta
import rasterio
from rasterio.transform import from_origin
from rasterio.features import geometry_mask
import time

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

countries_df = gpd.read_file('GIS/world_countries.shp')

Set the country you want to retrieve the images of ("kenya", "nigeria", "ethiopia" or "southafrica").

In [62]:
country='ethiopia'
win_len=2

country_codes={'kenya': 'KEN', 'nigeria': 'NIG', 'ethiopia': 'ETH', 'southafrica': 'SAF'}
country_fips={'kenya': 'KE', 'nigeria': 'NI', 'ethiopia': 'ET', 'southafrica': 'SF'}
country_years={'kenya': 2019, 'nigeria': 2020, 'ethiopia': 2019, 'southafrica': 2021}
country_months={'kenya': 8, 'nigeria': 1, 'ethiopia': 12, 'southafrica': 4}
country_names={'kenya': 'Kenya', 'nigeria': 'Nigeria', 'ethiopia': 'Ethiopia', 'southafrica': 'South Africa'}
country_code=country_codes[country]
country_fip=country_fips[country]
country_year=country_years[country]
country_month=country_months[country]
country_name=country_names[country]
month_start = (country_month % 12) + 1
year_start = country_year - (country_month!=12) - (win_len-1)

lta_start=2000

dest_folder = 'Afrobarometer/'+country_code+'/'
country_geometry = countries_df.loc[countries_df['COUNTRY']==country_name,:]['geometry'].to_crs(epsg=4326) 

country_ee = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co',country_fip)).first()


Temperature

In [63]:
temp_dataset = ee.ImageCollection('MODIS/061/MOD11A1').select(['LST_Day_1km'])
crs = temp_dataset.first().projection()
resolution = crs.nominalScale().getInfo()

first_date = datetime(year_start, month_start, 1, 0, 0)

date_list = ee.List([datetime(year, month_start, 1, 0, 0).strftime('%Y-%m-%d') for year in range(lta_start,year_start+1)])

#Map = folium.Map(location=[27.9506, -82.4572], zoom_start=8)# To see a google satellite view as a basemap
#Map.setOptions('HYBRID')

def year_mapper(date_str):
    startDate = ee.Date(date_str)
    endDate = startDate.advance(win_len, 'year')
    yearlyCollection = temp_dataset.filterDate(startDate, endDate)

    yearlyMean = yearlyCollection.mean();

    yearProperties = {
        'system:time_start': startDate.millis(),
        'system:time_end': endDate.millis()
    }

    return yearlyMean.set(yearProperties) 

yearAverages = ee.ImageCollection.fromImages(date_list.map(year_mapper))

ltaDate1 = ee.Date(date_list.get(0))
ltaDate2 = ee.Date(date_list.reverse().get(0)).advance(-win_len+1, 'year')
ltaAverages = yearAverages.filterDate(ltaDate1, ltaDate2)

ltaMean = ltaAverages.mean()

ltaStd = ltaAverages.reduce(ee.Reducer.stdDev())

lastImg = yearAverages.sort('system:time_start',False).first()

resImg = lastImg.subtract(ltaMean)
anomImg = resImg.divide(ltaStd)

anomImg_country = anomImg.clip(country_ee)

task_config = {
    'scale': resolution,  
    'region': country_ee.geometry()
    }

if(country=='kenya'):
    task_kenya_lst = ee.batch.Export.image(anomImg_country, country+'_LST_anoms', task_config)
    task_kenya_lst.start()
elif(country=='nigeria'):
    task_nigeria_lst = ee.batch.Export.image(anomImg_country, country+'_LST_anoms', task_config)
    task_nigeria_lst.start()
elif(country=='southafrica'):
    task_southafrica_lst = ee.batch.Export.image(anomImg_country, country+'_LST_anoms', task_config)
    task_southafrica_lst.start()
elif(country=='ethiopia'):
    task_ethiopia_lst = ee.batch.Export.image(anomImg_country, country+'_LST_anoms', task_config)
    task_ethiopia_lst.start()


Vegetation

In [64]:
ndvi_dataset = ee.ImageCollection("MODIS/061/MOD13Q1").select(['NDVI'])
crs = ndvi_dataset.first().projection()
resolution = crs.nominalScale().getInfo()

first_date = datetime(year_start, month_start, 1, 0, 0)

date_list = ee.List([datetime(year, month_start, 1, 0, 0).strftime('%Y-%m-%d') for year in range(lta_start,year_start+1)])

def year_mapper(date_str):
    startDate = ee.Date(date_str)
    endDate = startDate.advance(1, 'year')
    yearlyCollection = ndvi_dataset.filterDate(startDate, endDate)

    yearlyMean = yearlyCollection.mean();

    yearProperties = {
        'system:time_start': startDate.millis(),
        'system:time_end': endDate.millis()
    }

    return yearlyMean.set(yearProperties) 

yearAverages = ee.ImageCollection.fromImages(date_list.map(year_mapper))

ltaDate1 = ee.Date(date_list.get(0))
ltaDate2 = ee.Date(date_list.reverse().get(0)).advance(-win_len+1, 'year')
ltaAverages = yearAverages.filterDate(ltaDate1, ltaDate2)

ltaMean = ltaAverages.mean()

ltaStd = ltaAverages.reduce(ee.Reducer.stdDev())

lastImg = yearAverages.sort('system:time_start',False).first()

resImg = lastImg.subtract(ltaMean)
anomImg = resImg.divide(ltaStd)
anomImg_country = anomImg.clip(country_ee)

task_config = {
    'scale': resolution,  
    'region': country_ee.geometry(),
    'maxPixels': 200000000
    }

if(country=='kenya'):
    task_kenya_ndvi = ee.batch.Export.image(anomImg_country, country+'_NDVI_anoms', task_config)
    task_kenya_ndvi.start()
elif(country=='nigeria'):
    task_nigeria_ndvi = ee.batch.Export.image(anomImg_country, country+'_NDVI_anoms', task_config)
    task_nigeria_ndvi.start()
elif(country=='southafrica'):
    task_southafrica_ndvi = ee.batch.Export.image(anomImg_country, country+'_NDVI_anoms', task_config)
    task_southafrica_ndvi.start()
elif(country=='ethiopia'):
    task_ethiopia_ndvi = ee.batch.Export.image(anomImg_country, country+'_NDVI_anoms', task_config)
    task_ethiopia_ndvi.start()


Rainfall

In [65]:
nc_file_path='TAMSAT/'+country+'_rainfall.nc'

nc = netCDF4.Dataset(nc_file_path, 'r')
time_values = nc.variables['time'][:]
year_values = np.array([date.fromtimestamp(time_value).year for time_value in time_values])
month_values = np.array([date.fromtimestamp(time_value).month for time_value in time_values])

lon = nc.variables['lon'][:]
lat = nc.variables['lat'][:]
pixel_size_x = abs(lon[1] - lon[0])
pixel_size_y = abs(lat[1] - lat[0])

upper_left_x = min(lon)
upper_left_y = max(lat)

transform = from_origin(upper_left_x, upper_left_y, pixel_size_x, pixel_size_y)

data_values = np.array(nc.variables['rfe'][:])

yearly_rasters = np.zeros((year_start+1-lta_start,len(lat),len(lon)))

for year in range(lta_start,year_start+1):
    
    time_mask = np.flatnonzero(np.logical_and(month_values==month_start, year_values==year))[0]+np.arange(win_len*12,dtype=int)
    
    yearly_raster = np.sum(data_values[time_mask,:,:], axis=0)
    
    yearly_rasters[year-lta_start,:,:]=yearly_raster

mean_raster = np.mean(yearly_rasters[:-win_len,:,:], axis=0)
std_raster = np.std(yearly_rasters[:-win_len,:,:], axis=0)

afb_raster = yearly_rasters[-1,:,:]

anom_raster = (afb_raster - mean_raster)/std_raster

poly_mask = geometry_mask(country_geometry, out_shape=(len(lat), len(lon)), transform=transform, invert=False)

masked_data = np.ma.masked_array(anom_raster, poly_mask)


with rasterio.open(
    'Maps/'+country+'_rfe_anoms.tif',
    "w",
    driver="GTiff",
    width=masked_data.shape[1],
    height=masked_data.shape[0],
    count=1,
    dtype=masked_data.dtype,
    crs='EPSG:4326',
    transform=transform,
    nodata=np.nan
) as dst:
    dst.write(masked_data, indexes=1)

74    POLYGON ((37.95210 14.83755, 37.95538 14.83652...
Name: geometry, dtype: geometry


/home/mlissoni/.local/lib/python3.10/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/tmp/ipykernel_23372/1922744105.py:36: RuntimeWarning: invalid value encountered in subtract
  anom_raster = (afb_raster - mean_raster)/std_raster


The tasks on Google Earth Engine can take some time. Check their status here.

In [70]:
task_kenya_lst.status()

{'state': 'RUNNING',
 'description': 'kenya_LST_anoms',
 'creation_timestamp_ms': 1710822026780,
 'update_timestamp_ms': 1710822451374,
 'start_timestamp_ms': 1710822031726,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'F7R73ZLS4T4TUOT3X2SZVADB',
 'name': 'projects/earthengine-legacy/operations/F7R73ZLS4T4TUOT3X2SZVADB'}